In [1]:
IS_RENDER = 0
TCP_PORT = 13001 # ur3, 1DoF

In [2]:
from mujoco_py import load_model_from_xml, load_model_from_path, MjSim, MjViewer
import math
import os
import numpy as np

from mujoco_py.modder import TextureModder

class manipulator():
    def __init__(self):
        
        self.DoF = DoF = 1
        self.model = load_model_from_path("model-ur3/ur3-3.xml")
        self.sim = MjSim(self.model)
        
        self.sim.data.qpos[1] = -np.pi/2         
        self.sim.data.qpos[3] = -np.pi/2
        self.sim.step()

        if IS_RENDER: self.viewer = MjViewer(self.sim)
        self.sim_state = self.sim.get_state()
        
        self.state_dim = DoF*3+3
        self.action_dim = DoF
        
    def reset(self):
        self.sim.set_state(self.sim_state)
        
        rxyz = np.array([ -0.2986, 0.19425, 0.39555 ])
        self.rx, self.ry, self.rz = rxyz[0], rxyz[1], rxyz[2]
        
        self.qvel=np.zeros( self.DoF )
        
        s = []
        for i in range(self.DoF):
            s.append( self.qvel[i] )
        for i in [2]:
            s.append( self.sim.data.qvel[i] )
        for i in [2]:
            s.append( self.sim.data.qpos[i] )
        s.append( self.rx )
        s.append( self.ry )
        s.append( self.rz )
        s = np.array(s)
        
        return s
    def step(self, action):
        for i in range( self.DoF ):
            self.qvel[i] += action[i]

#             v_lmt = 1.0
#             if self.qvel[i]>v_lmt:
#                 self.qvel[i] = v_lmt
#             if self.qvel[i] < -v_lmt:
#                 self.qvel[i] = -v_lmt
    
        self.sim.data.qvel[2] = self.qvel[0]
        
        for i in [0,1,3,4,5]:
            self.sim.data.qvel[i] = 0
            self.sim.data.qpos[i] = 0
            if i == 1:
                self.sim.data.qpos[1] = -np.pi/2         
            if i == 3:
                self.sim.data.qpos[3] = -np.pi/2
        
        self.sim.step()
        if IS_RENDER: self.viewer.render()
        
        # print("self.sim.data.sensordata: ", self.sim.data.sensordata)
        dis = np.linalg.norm(self.sim.data.sensordata[-3:]-[self.rx, self.ry, self.rz])
        r = -dis

        s = []
        for i in range(self.DoF):
            s.append( self.qvel[i] )
        for i in [2]:
            s.append( self.sim.data.qvel[i] )
        for i in [2]:
            s.append( self.sim.data.qpos[i] )
        s.append( self.rx )
        s.append( self.ry )
        s.append( self.rz )
        s = np.array(s)
        
        d = 0
        info = [dis]
        return s, r, d, info

## 1.2 Core Code of Servicer:

In [3]:
import numpy as np

import socket
import threading

import json

def env_servicer(env):
    #Create The Socket
    s=socket.socket(socket.AF_INET,socket.SOCK_STREAM)

    #Listen The Port
    s.bind(('',TCP_PORT))
    s.listen(5)
    print( 'TCP_PORT: ', TCP_PORT, 'Waiting for connection...')

    def tcplink(sock,addr):
        print('Accept new connection from %s:%s...' % addr)
        sock.send( ('Welcome!').encode() )
        while True:
            data=sock.recv(1024).decode()
            # print("data: ", data)

            data_json = json.loads( data )
            # print('data_json["type"]: ', data_json["type"])

            if data_json["type"] == "init":
                data = { 'state_dim' : env.state_dim, 'action_dim' : env.action_dim, 'DoF' : env.DoF } 

            elif data_json["type"] == "reset":
                state = env.reset()
                data = { 'state' : state.tolist() } 

            elif data_json["type"] == "step":
                a = np.array( data_json["action"] )
                state_next, r, done, info = env.step(a)
                data = { 'state' : state_next.tolist(), 'reward' : r, 'done' : done, 'info' : info } 
            elif data_json["type"] == "close":
                break
                
            str_json = json.dumps(data)
            sock.send( str_json.encode() )
        sock.close()
        print('Connection from %s:%s closed.'%addr)
        
    while True:
        # Accept A New Connection
        sock,addr=s.accept()
        
        # Create A New Thread to Deal with The TCP Connection
        t=threading.Thread(target=tcplink(sock,addr))

In [ ]:
env_servicer( manipulator() )

TCP_PORT:  13001 Waiting for connection...
Accept new connection from 192.168.0.10:49785...
Connection from 192.168.0.10:49785 closed.
Accept new connection from 192.168.0.10:49805...
